## 7.4　訓練ループと評価ループを独自に作成する

In [1]:
import copy
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

#### 7.4.1　訓練と推論

In [2]:
def train_step(inputs, targets):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(model.trainable_weights, gradients))

#### 7.4.2　指標の低レベルの使い方

In [3]:
metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f'result: {current_result:.2f}')

result: 1.00


In [4]:
values = [0, 1, 2, 3, 4]
mean_tracker = keras.metrics.Mean()
for value in values:
  mean_tracker.update_state(value)
print(f'Mean of values: {mean_tracker.result():.2f}')

Mean of values: 2.00


#### 7.4.3　完全な訓練ループと評価ループ

In [5]:
def get_mnist_model():
  inputs = keras.Input(shape=(28*28,))
  features = layers.Dense(512, activation='relu')(inputs)
  features = layers.Dropout(0.5)(features)
  outputs = layers.Dense(10, activation='softmax')(features)
  model = keras.Model(inputs, outputs)
  return model

In [6]:
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype('float32') / 255
test_images = test_images.reshape((10000, 28*28)).astype('float32') / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [7]:
model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

In [8]:
def train_step(inputs, targets):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    loss = loss_fn(targets, predictions)
  gradients = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs[metric.name] = metric.result()

  loss_tracking_metric.update_state(loss)
  logs['loss'] = loss_tracking_metric.result()
  return logs

def reset_metrics():
  for metric in metrics:
    metric.reset_state()
  loss_tracking_metric.reset_state()

In [9]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.batch(32)
epochs = 3

for epoch in range(epochs):
  reset_metrics()
  for inputs_batch, targets_batch in training_dataset:
    logs = train_step(inputs_batch, targets_batch)
  print(f'Results at the end of epoch {epoch}')
  for key, value in logs.items():
    print(f'...{key}: {value:.4f}')

Results at the end of epoch 0
...sparse_categorical_accuracy: 0.9129
...loss: 0.2895
Results at the end of epoch 1
...sparse_categorical_accuracy: 0.9543
...loss: 0.1639
Results at the end of epoch 2
...sparse_categorical_accuracy: 0.9635
...loss: 0.1287


In [10]:
def test_step(inputs, targets):
  predictions = model(inputs, training=False)
  loss = loss_fn(targets, predictions)

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs['val_' + metric.name] = metric.result()

  loss_tracking_metric.update_state(loss)
  logs['val_loss'] = loss_tracking_metric.result()
  return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()

for inputs_batch, targets_batch in val_dataset:
  logs = test_step(inputs_batch, targets_batch)
print('Evaluation results:')
for key, value in logs.items():
  print(f'...{key}: {value:.4f}')

Evaluation results:
...val_sparse_categorical_accuracy: 0.9677
...val_loss: 0.1190


#### 7.4.4　tf.function による高速化

In [11]:
@tf.function
def test_step(inputs, targets):
  predictions = model(inputs, training=False)
  loss = loss_fn(targets, predictions)

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs['val_' + metric.name] = metric.result()

  loss_tracking_metric.update_state(loss)
  logs['val_loss'] = loss_tracking_metric.result()
  return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()

for inputs_batch, targets_batch in val_dataset:
  logs = test_step(inputs_batch, targets_batch)
print('Evaluation results:')
for key, value in logs.items():
  print(f'...{key}: {value:.4f}')

Evaluation results:
...val_sparse_categorical_accuracy: 0.9677
...val_loss: 0.1190


#### 7.4.5　カスタム訓練ループで fit( ) を活用する

In [12]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name='loss')

class CustomModel(keras.Model):
  def train_step(self, data):
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    self.optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    loss_tracker.update_state(loss)
    return {'loss': loss_tracker.result()}

  @property
  def metrics(self):
    return [loss_tracker]

In [13]:
inputs = keras.Input(shape=(28*28, ))
features = layers.Dense(512, activation='relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation='softmax')(features)

model = CustomModel(inputs, outputs)
model.compile(optimizer=keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.2991
Epoch 2/3
1563/1563 [==============================] - 8s 5ms/step - loss: 0.1629
Epoch 3/3
1563/1563 [==============================] - 9s 6ms/step - loss: 0.1327


In [14]:
class CustomModel(keras.Model):
  def train_step(self, data):
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.compiled_loss(targets, predictions)
    gradients = tape.gradient(loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
    self.compiled_metrics.update_state(targets, predictions)
    return {m.name: m.result() for m in self.metrics}

In [15]:
inputs = keras.Input(shape=(28*28, ))
features = layers.Dense(512, activation='relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation='softmax')(features)

model = CustomModel(inputs, outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.2949 - sparse_categorical_accuracy: 0.9121
Epoch 2/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.1620 - sparse_categorical_accuracy: 0.9541
Epoch 3/3
1563/1563 [==============================] - 16s 10ms/step - loss: 0.1323 - sparse_categorical_accuracy: 0.9632
